In [2]:
from pathlib import Path

import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Figure out where this notebook is running
NOTEBOOK_DIR = Path.cwd()
print("CWD:", NOTEBOOK_DIR)

# If we're in .../sammo-fight-iq/notebooks, project root is the parent
if NOTEBOOK_DIR.name == "notebooks":
    PROJECT_ROOT = NOTEBOOK_DIR.parent
else:
    PROJECT_ROOT = NOTEBOOK_DIR

DATA_DIR = PROJECT_ROOT / "data"
VIDEO_PATH = DATA_DIR / "round1.mp4"

print("Video path:", VIDEO_PATH)
print("Exists:", VIDEO_PATH.exists())

CWD: /opt/app-root/src/sammo-fight-iq
Video path: /opt/app-root/src/sammo-fight-iq/data/round1.mp4
Exists: True


In [3]:
mp_pose = mp.solutions.pose

def frame_boxing_metrics(landmarks, mp_pose=mp_pose):
    """Compute basic boxing metrics from one frame using pose landmarks."""
    lmk = landmarks

    left_shoulder  = lmk[mp_pose.PoseLandmark.LEFT_SHOULDER]
    right_shoulder = lmk[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    left_wrist     = lmk[mp_pose.PoseLandmark.LEFT_WRIST]
    right_wrist    = lmk[mp_pose.PoseLandmark.RIGHT_WRIST]
    left_hip       = lmk[mp_pose.PoseLandmark.LEFT_HIP]
    right_hip      = lmk[mp_pose.PoseLandmark.RIGHT_HIP]
    nose           = lmk[mp_pose.PoseLandmark.NOSE]

    # Guard height = wrist relative to shoulder (lower = tighter guard)
    left_guard_height  = left_wrist.y  - left_shoulder.y
    right_guard_height = right_wrist.y - right_shoulder.y

    # Simple hip rotation + stance width (x-distance between hips)
    hip_rotation = abs(left_hip.x - right_hip.x)
    stance_width = abs(left_hip.x - right_hip.x)

    return {
        "left_guard_height":  left_guard_height,
        "right_guard_height": right_guard_height,
        "head_y":             nose.y,
        "hip_rotation":       hip_rotation,
        "stance_width":       stance_width,
    }

print("Helper ready.")

Helper ready.


In [4]:
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

cap = cv2.VideoCapture(str(VIDEO_PATH))
if not cap.isOpened():
    raise RuntimeError(f"Could not open video: {VIDEO_PATH}")

frame_idx = 0
rows = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1

    # BGR → RGB for Mediapipe
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    if results.pose_landmarks:
        metrics = frame_boxing_metrics(results.pose_landmarks.landmark)
        metrics["pose_detected"] = True
    else:
        metrics = {
            "left_guard_height": np.nan,
            "right_guard_height": np.nan,
            "head_y": np.nan,
            "hip_rotation": np.nan,
            "stance_width": np.nan,
            "pose_detected": False,
        }

    metrics["frame"] = frame_idx
    rows.append(metrics)

cap.release()
pose.close()

frame_df = pd.DataFrame(rows)
print("Frames processed:", len(frame_df))
frame_df.head()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1763222430.895003    1961 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1763222430.933193    1961 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/opt/app-root/lib64/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Frames processed: 14095


,left_guard_height,right_guard_height,head_y,hip_rotation,stance_width,pose_detected,frame
0,NaN,NaN,NaN,NaN,NaN,False,1
1,NaN,NaN,NaN,NaN,NaN,False,2
2,NaN,NaN,NaN,NaN,NaN,False,3
3,0.014106,0.037065,0.128473,0.001398,0.001398,True,4
4,-0.051068,0.010931,0.142394,0.044306,0.044306,True,5


In [5]:
df = frame_df.copy()

total_frames = len(df)
pose_frames = int(df["pose_detected"].sum())

# treat guard as "down" if wrist is a fair bit below shoulder
guard_down_thresh = 0.15
guard_down_frames = int(
    ((df["left_guard_height"] > guard_down_thresh) |
     (df["right_guard_height"] > guard_down_thresh)).sum()
)

round_stats = {
    "round_id": "round1",  # later we’ll make this dynamic
    "total_frames": total_frames,
    "pose_frames": pose_frames,
    "pose_coverage": pose_frames / total_frames if total_frames else 0.0,
    "guard_down_ratio": guard_down_frames / pose_frames if pose_frames else 0.0,
    "avg_left_guard_height":  float(df["left_guard_height"].mean()),
    "avg_right_guard_height": float(df["right_guard_height"].mean()),
    "avg_hip_rotation":       float(df["hip_rotation"].mean()),
    "avg_stance_width":       float(df["stance_width"].mean()),
    "avg_head_y":             float(df["head_y"].mean()),
}

pd.Series(round_stats)


round_id                    round1
total_frames                 14095
pose_frames                   8520
pose_coverage              0.60447
guard_down_ratio          0.010329
avg_left_guard_height     0.031025
avg_right_guard_height     0.03553
avg_hip_rotation           0.04255
avg_stance_width           0.04255
avg_head_y                0.630725
dtype: object

In [6]:
def video_form_and_danger(stats: dict) -> dict:
    guard_down = stats["guard_down_ratio"]          # 0–1
    pose_cov   = stats["pose_coverage"]            # 0–1

    # higher danger if guard is down + low pose coverage
    danger = 0.6 * guard_down + 0.4 * (1 - pose_cov)
    danger = max(0.0, min(1.0, danger))

    # form score starts at 10, subtract penalties
    form = 10.0
    form -= guard_down * 5.0       # big hit if guard down a lot
    form -= (1 - pose_cov) * 2.0   # small hit if tracking low
    form = max(0.0, min(10.0, form))

    if danger >= 0.7:
        focus = "defense_first"
    elif danger >= 0.4:
        focus = "ring_cutting"
    else:
        focus = "pressure_and_body"

    out = stats.copy()
    out["video_danger_score"] = float(danger)
    out["video_form_score"] = float(form)
    out["video_focus_next_round"] = focus
    return out

round_enriched = video_form_and_danger(round_stats)
pd.Series(round_enriched)


round_id                             round1
total_frames                          14095
pose_frames                            8520
pose_coverage                       0.60447
guard_down_ratio                   0.010329
avg_left_guard_height              0.031025
avg_right_guard_height              0.03553
avg_hip_rotation                    0.04255
avg_stance_width                    0.04255
avg_head_y                         0.630725
video_danger_score                 0.164409
video_form_score                   9.157296
video_focus_next_round    pressure_and_body
dtype: object

In [7]:
import json
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
OUT_DATA = PROJECT_ROOT / "data"
OUT_DATA.mkdir(parents=True, exist_ok=True)

# CSV of all rounds
csv_path = OUT_DATA / "video_round_stats.csv"
round_df = pd.DataFrame([round_enriched])

if csv_path.exists():
    existing = pd.read_csv(csv_path)
    combined = pd.concat([existing, round_df], ignore_index=True)
else:
    combined = round_df

combined.to_csv(csv_path, index=False)
print("✅ Updated CSV:", csv_path)

# JSON (one per round) for ai_stats-style usage
AI_STATS_DIR = PROJECT_ROOT / "ai_stats"
AI_STATS_DIR.mkdir(parents=True, exist_ok=True)

json_path = AI_STATS_DIR / f"{round_enriched['round_id']}.json"
with json_path.open("w") as f:
    json.dump(round_enriched, f, indent=2)

print("✅ Saved JSON:", json_path)
round_enriched


✅ Updated CSV: /opt/app-root/src/data/video_round_stats.csv
✅ Saved JSON: /opt/app-root/src/ai_stats/round1.json


{'round_id': 'round1',
 'total_frames': 14095,
 'pose_frames': 8520,
 'pose_coverage': 0.6044696700957787,
 'guard_down_ratio': 0.010328638497652582,
 'avg_left_guard_height': 0.031024675537338,
 'avg_right_guard_height': 0.03552993167751412,
 'avg_hip_rotation': 0.0425502892426202,
 'avg_stance_width': 0.0425502892426202,
 'avg_head_y': 0.6307253441025673,
 'video_danger_score': 0.1644093150602801,
 'video_form_score': 9.157296147703295,
 'video_focus_next_round': 'pressure_and_body'}

In [8]:
from src.risk_model import video_form_and_danger